### Матричные факторизации

В данной работе вам предстоит познакомиться с практической стороной матричных разложений.
Работа поделена на 4 задания:
1. Вам необходимо реализовать SVD разложения используя SGD на explicit данных
2. Вам необходимо реализовать матричное разложения используя ALS на implicit данных
3. Вам необходимо реализовать матричное разложения используя BPR(pair-wise loss) на implicit данных
4. Вам необходимо реализовать матричное разложения используя WARP(list-wise loss) на implicit данных

Мягкий дедлайн 28 Сентября (пишутся замечания, выставляется оценка, есть возможность исправить до жесткого дедлайна)

Жесткий дедлайн 5 Октября (Итоговая проверка)

In [1]:
import implicit
import pandas as pd
import numpy as np
import scipy.sparse as sp

c:\users\user\appdata\local\programs\python\python36\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
c:\users\user\appdata\local\programs\python\python36\lib\site-packages\numpy\.libs\libopenblas.noijjg62emaszi6nyurl6jbkm4evbgm7.gfortran-win_amd64.dll
c:\users\user\appdata\local\programs\python\python36\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


В данной работе мы будем работать с explicit датасетом movieLens, в котором представленны пары user_id movie_id и rating выставленный пользователем фильму

Скачать датасет можно по ссылке https://grouplens.org/datasets/movielens/1m/

In [2]:
ratings = pd.read_csv('ml-1m/ratings.dat', delimiter='::', header=None, 
        names=['user_id', 'movie_id', 'rating', 'timestamp'], 
        usecols=['user_id', 'movie_id', 'rating'], engine='python')

In [3]:
ratings['movie_id'].drop_duplicates()

0         1193
1          661
2          914
3         3408
4         2355
          ... 
919876    2198
940262    2703
957826    2845
970914    3607
983062    2909
Name: movie_id, Length: 3706, dtype: int64

In [4]:
movie_info = pd.read_csv('ml-1m/movies.dat', delimiter='::', header=None, 
        names=['movie_id', 'name', 'category'], engine='python')

Explicit данные

In [5]:
ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
5,1,1197,3
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4


Для того, чтобы преобразовать текущий датасет в Implicit, давайте считать что позитивная оценка это оценка >=4

In [6]:
implicit_ratings = ratings.loc[(ratings['rating'] >= 4)]

In [7]:
implicit_ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
3,1,3408,4
4,1,2355,5
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4
10,1,595,5
11,1,938,4
12,1,2398,4


Удобнее работать с sparse матричками, давайте преобразуем DataFrame в CSR матрицы

In [8]:
users = implicit_ratings["user_id"]
movies = implicit_ratings["movie_id"]
user_item = sp.coo_matrix((np.ones_like(users), (users, movies)))
user_item_t_csr = user_item.T.tocsr()
user_item_csr = user_item.tocsr()

В качестве примера воспользуемся ALS разложением из библиотеки implicit

Зададим размерность латентного пространства равным 64, это же определяет размер user/item эмбедингов

In [9]:
toy_movie_id = 1
test_user_id = 4

In [25]:
model = implicit.als.AlternatingLeastSquares(factors=64, iterations=100, calculate_training_loss=True)

В качестве loss здесь всеми любимый RMSE

In [26]:
model.fit(user_item_t_csr)

Построим похожие фильмы по 1 movie_id = Истории игрушек

In [27]:
movie_info.head(5)

,movie_id,name,category
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [28]:
get_similars = lambda item_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                        for x in model.similar_items(item_id)]

Как мы видим, симилары действительно оказались симиларами.

Качество симиларов часто является хорошим способом проверить качество алгоритмов.

P.S. Если хочется поглубже разобраться в том как разные алгоритмы формируют разные латентные пространства, рекомендую загружать полученные вектора в tensorBoard и смотреть на сформированное пространство

In [29]:
get_similars(1, model)

['0    Toy Story (1995)',
 '3045    Toy Story 2 (1999)',
 "2286    Bug's Life, A (1998)",
 '33    Babe (1995)',
 '2315    Babe: Pig in the City (1998)',
 '584    Aladdin (1992)',
 '1526    Hercules (1997)',
 '2252    Pleasantville (1998)',
 '1838    Mulan (1998)',
 '2692    Iron Giant, The (1999)']

Давайте теперь построим рекомендации для юзеров

Как мы видим юзеру нравится фантастика, значит и в рекомендациях ожидаем увидеть фантастику

In [30]:
get_user_history = lambda user_id, implicit_ratings : [movie_info[movie_info["movie_id"] == x]["name"].to_string() 
                                            for x in implicit_ratings[implicit_ratings["user_id"] == user_id]["movie_id"]]

In [31]:
get_user_history(4, implicit_ratings)

['3399    Hustler, The (1961)',
 '2882    Fistful of Dollars, A (1964)',
 '1196    Alien (1979)',
 '1023    Die Hard (1988)',
 '257    Star Wars: Episode IV - A New Hope (1977)',
 '1959    Saving Private Ryan (1998)',
 '476    Jurassic Park (1993)',
 '1180    Raiders of the Lost Ark (1981)',
 '1885    Rocky (1976)',
 '1081    E.T. the Extra-Terrestrial (1982)',
 '3349    Thelma & Louise (1991)',
 '3633    Mad Max (1979)',
 '2297    King Kong (1933)',
 '1366    Jaws (1975)',
 '1183    Good, The Bad and The Ugly, The (1966)',
 '2623    Run Lola Run (Lola rennt) (1998)',
 '2878    Goldfinger (1964)',
 '1220    Terminator, The (1984)']

Получилось! 

Мы действительно порекомендовали пользователю фантастику и боевики, более того встречаются продолжения тех фильмов, которые он высоко оценил

In [32]:
get_recommendations = lambda user_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                               for x in model.recommend(user_id, user_item_csr)]

In [33]:
get_recommendations(4, model)

['585    Terminator 2: Judgment Day (1991)',
 '1271    Indiana Jones and the Last Crusade (1989)',
 '1178    Star Wars: Episode V - The Empire Strikes Back...',
 '1284    Butch Cassidy and the Sundance Kid (1969)',
 '1182    Aliens (1986)',
 '2502    Matrix, The (1999)',
 '1884    French Connection, The (1971)',
 '3402    Close Encounters of the Third Kind (1977)',
 '3458    Predator (1987)',
 '1892    Rain Man (1988)']

Теперь ваша очередь реализовать самые популярные алгоритмы матричных разложений

Что будет оцениваться:
1. Корректность алгоритма
2. Качество получившихся симиларов
3. Качество итоговых рекомендаций для юзера

### Задание 1. Не использую готовые решения, реализовать SVD разложение используя SGD на explicit данных

In [119]:
class SVD_Recommender:

    def fit(self, ratings, features_dim=64, learning_rate=0.01, lambda_reg=0.01):
        height = max(ratings['user_id']) + 1
        W = np.random.random((height, features_dim)) / np.sqrt(features_dim)
        bias_w = np.zeros(height)

        width = max(ratings['movie_id']) + 1
        H = np.random.random((width, features_dim)) / np.sqrt(features_dim)
        bias_h = np.zeros(width)

        eps = 1e-2
        ratings = ratings.to_numpy()
        for _ in range(10):
            total_error = 0
            np.random.shuffle(ratings)
            for row in ratings:
                user_id = row[0]
                item_id = row[1]

                error = np.dot(W[user_id], H[item_id]) + bias_w[user_id] + bias_h[item_id] - row[2]
                total_error += error ** 2

                W[user_id] = W[user_id] - learning_rate * (error * H[item_id] + lambda_reg * W[user_id])
                bias_w[user_id] = bias_w[user_id] - learning_rate * (error + lambda_reg * bias_w[user_id])

                H[item_id] = H[item_id] - learning_rate * (error * W[user_id] + lambda_reg * H[item_id])
                bias_h[item_id] = bias_h[item_id] - learning_rate * (error + lambda_reg * bias_h[item_id])

            mse = total_error / len(ratings)
            print('mse', mse)

            if mse < eps:
                break
                
        self.W = W
        self.bias_w = bias_w
        self.H = H
        self.bias_h = bias_h

In [120]:
svd = SVD_Recommender()
svd.fit(ratings)

mse 1.4790943724404508
mse 0.8627681368845151
mse 0.8327983734999971
mse 0.8065620175469181
mse 0.7693825776816552
mse 0.7269893828026432
mse 0.6831431616879166
mse 0.6401102806602547
mse 0.5992764155656694
mse 0.5620845429846378


In [10]:
def similars(recommender, movie_id):
    input_vector = recommender.H[movie_id]

    data = []
    for item_idx, column in enumerate(recommender.H):
        dst = np.linalg.norm(column - input_vector)
        data.append((item_idx, dst))

    sorted_by_dst = list(sorted(data, key=lambda val: val[1]))

    similars = []
    for item in sorted_by_dst:
        search = movie_info[movie_info["movie_id"] == item[0]]
        movie_name = search["name"].to_string()
        if len(search) > 0:
            similars.append((item[0], movie_name))

    return similars

def recommend(recommender, user_id, user_item_csr):
    movie_ids = user_item_csr[user_id].indices

    data = []
    for movie_id in movie_ids:
        if movie_id >= len(recommender.H):
            continue
            
        bias_w = 0
        if hasattr(recommender, 'bias_w'):
            bias_w = recommender.bias_w[user_id]
            
        bias_h = 0
        if hasattr(recommender, 'bias_h'):
            bias_h = recommender.bias_h[movie_id]

        dot = np.dot(recommender.W[user_id], recommender.H[movie_id])
        data.append((movie_id, dot + bias_w + bias_h))

    data = list(sorted(data, key=lambda val: val[1], reverse=True))
    recommendations = []
    for x in data:
         recommendations.append(movie_info[movie_info["movie_id"] == x[0]]["name"].to_string())
    return recommendations

In [124]:
similars(svd, toy_movie_id)[:10]

[(1, '0    Toy Story (1995)'),
 (3114, '3045    Toy Story 2 (1999)'),
 (2355, "2286    Bug's Life, A (1998)"),
 (2294, '2225    Antz (1998)'),
 (1713, '1664    Mouse Hunt (1997)'),
 (239, '236    Goofy Movie, A (1995)'),
 (1907, '1838    Mulan (1998)'),
 (2687, '2618    Tarzan (1999)'),
 (3615, '3546    Dinosaur (2000)'),
 (709, '700    Oliver & Company (1988)')]

In [123]:
recommend(svd, test_user_id, user_item_csr)[:10]

['3399    Hustler, The (1961)',
 '2882    Fistful of Dollars, A (1964)',
 '1366    Jaws (1975)',
 '1959    Saving Private Ryan (1998)',
 '1183    Good, The Bad and The Ugly, The (1966)',
 '1885    Rocky (1976)',
 '1180    Raiders of the Lost Ark (1981)',
 '2878    Goldfinger (1964)',
 '1081    E.T. the Extra-Terrestrial (1982)',
 '1220    Terminator, The (1984)']

### Задание 2. Не использую готовые решения, реализовать матричное разложение используя ALS на implicit данных

In [11]:
from tqdm.notebook import tqdm

class ALS_Recommender:

    def fit(self, input_matrix, features_dim=64, lambda_reg=0.001, alpha_conf=40, max_iterations=2):
        height = max(ratings['user_id']) + 1
        W = np.random.random((height, features_dim)) / np.sqrt(features_dim)

        width = max(ratings['movie_id']) + 1
        H = np.random.random((width, features_dim)) / np.sqrt(features_dim)

        while iteration < max_iterations:
            HTH = np.dot(H.T, H)
            for user_id in tqdm(range(height)):
                p_u = input_matrix[user_id, :].toarray().squeeze()
                c = 1 + alpha_conf * p_u
                Cu = np.diag(p_u)
                
                triple_dot = H.T.dot(Cu - np.eye(width)).dot(H)
                inversed = np.linalg.inv(HTH + triple_dot + eye_reg)
                right_mult =  H.T.dot(Cu).dot(p_u)
                W[user_id] = np.dot(inversed, right_mult)

            WTW = np.dot(W.T, W)
            for item_id in tqdm(range(width)):
                p_i = input_matrix[:, item_id].toarray().squeeze()
                c = 1 + alpha_conf * p_i
                Cu = np.diag(p_i)

                triple_dot = W.T.dot(Cu - np.eye(height)).dot(W) 
                inversed = np.linalg.inv(WTW + triple_dot + eye_reg)
                right_mult =  W.T.dot(Cu).dot(p_i)
                H[item_id] = np.dot(inversed, right_mult)


            iteration += 1

        self.W = W
        self.H = H

In [ ]:
als = ALS_Recommender()
als.fit(user_item_csr)

In [178]:
similars(als, toy_movie_id)[:10]

[(1, '0    Toy Story (1995)'),
 (3114, '3045    Toy Story 2 (1999)'),
 (588, '584    Aladdin (1992)'),
 (364, '360    Lion King, The (1994)'),
 (1907, '1838    Mulan (1998)'),
 (2761, '2692    Iron Giant, The (1999)'),
 (1265, '1245    Groundhog Day (1993)'),
 (2687, '2618    Tarzan (1999)'),
 (1566, '1526    Hercules (1997)'),
 (2294, '2225    Antz (1998)')]

In [186]:
recommend(als, test_user_id, user_item_csr)[:10]

['257    Star Wars: Episode IV - A New Hope (1977)',
 '1180    Raiders of the Lost Ark (1981)',
 '1366    Jaws (1975)',
 '1196    Alien (1979)',
 '1220    Terminator, The (1984)',
 '1023    Die Hard (1988)',
 '1081    E.T. the Extra-Terrestrial (1982)',
 '1183    Good, The Bad and The Ugly, The (1966)',
 '1959    Saving Private Ryan (1998)',
 '1885    Rocky (1976)']

### Задание 3. Не использую готовые решения, реализовать матричное разложение BPR на implicit данных

In [171]:
from tqdm.notebook import tqdm
import random

class BPR_recommender():
    
    def fit(self, input_matrix, features_dim=64, lambda_reg=0.000001, learning_rate=0.05, max_iterations=5):
        height = max(ratings['user_id']) + 1
        W = np.random.random((height, features_dim)) / np.sqrt(features_dim)

        width = max(ratings['movie_id']) + 1
        H = np.random.random((width, features_dim)) / np.sqrt(features_dim)
        
        Ds = []
        for user_id in ratings['user_id'].unique():
            for movie_id in user_item_csr[user_id, :].indices:
                Ds.append((user_id, movie_id))

        for epoch in tqdm(range(max_iterations)):       
            np.random.shuffle(Ds)

            for user_id, i in tqdm(Ds):
                j = np.random.randint(1, width)
                while input_matrix[user_id, j] != 0:
                    j = np.random.randint(1, width)

                x_hat_uij = np.dot(W[user_id], H[i]) - np.dot(W[user_id], H[j])

                coef = np.exp(-x_hat_uij) / (1 + np.exp(-x_hat_uij))
                W[user_id] = W[user_id] + learning_rate * (coef * (H[i] - H[j]) + lambda_reg * W[user_id])
                H[i] = H[i] + learning_rate * (coef * (W[user_id]) + lambda_reg * H[i])
                H[j] = H[j] + learning_rate * (coef * (-W[user_id]) + lambda_reg * H[j])

        self.W = W
        self.H = H

In [172]:
bpr = BPR_recommender()
bpr.fit(user_item_csr, max_iterations=20)

In [173]:
similars(bpr, toy_movie_id)[:10]

[(1, '0    Toy Story (1995)'),
 (3114, '3045    Toy Story 2 (1999)'),
 (2687, '2618    Tarzan (1999)'),
 (364, '360    Lion King, The (1994)'),
 (1907, '1838    Mulan (1998)'),
 (551, '547    Nightmare Before Christmas, The (1993)'),
 (34, '33    Babe (1995)'),
 (661, '655    James and the Giant Peach (1996)'),
 (2294, '2225    Antz (1998)'),
 (1566, '1526    Hercules (1997)')]

In [174]:
recommend(bpr, test_user_id, user_item_csr)[:10]

['257    Star Wars: Episode IV - A New Hope (1977)',
 '1180    Raiders of the Lost Ark (1981)',
 '1959    Saving Private Ryan (1998)',
 '1220    Terminator, The (1984)',
 '1196    Alien (1979)',
 '1366    Jaws (1975)',
 '1023    Die Hard (1988)',
 '3349    Thelma & Louise (1991)',
 '1885    Rocky (1976)',
 '476    Jurassic Park (1993)']

### Задание 4. Не использую готовые решения, реализовать матричное разложение WARP на implicit данных

In [46]:
from tqdm.notebook import tqdm

class WARP_recommender():
    
    def fit(self, input_matrix, features_dim=64, lambda_reg=0.00001, learning_rate=0.05, max_iterations=10, margin=0.6):
        height = max(ratings['user_id']) + 1
        W = np.random.random((height, features_dim)) / np.sqrt(features_dim)

        width = max(ratings['movie_id']) + 1
        H = np.random.random((width, features_dim)) / np.sqrt(features_dim)
        
        Ds = []
        for user_id in ratings['user_id'].unique():
            for movie_id in user_item_csr[user_id, :].indices:
                Ds.append((user_id, movie_id))
        losses = []
        for epoch in tqdm(range(max_iterations)):       
            np.random.shuffle(Ds)

            for user_id, i in tqdm(Ds):
                for n in range(1, 10):
                    j = np.random.randint(1, width)
                    total_neg_samples = 1
                    while input_matrix[user_id, j] != 0:
                        j = np.random.randint(1, width)

                    if margin + np.dot(W[user_id], H[j]) > np.dot(W[user_id], H[i]):
                        loss = np.log(10 / n)
                        W[user_id] = W[user_id] - learning_rate * (loss * (H[j] + H[i]) + lambda_reg * W[user_id])
                        H[i] = H[i] - learning_rate * (loss * (-W[user_id]) + lambda_reg * H[i])
                        H[j] = H[j] - learning_rate * (loss * (W[user_id]) + lambda_reg * H[j])
                        
        self.W = W
        self.H = H

In [47]:
warp = WARP_recommender()
warp.fit(user_item_csr)

In [48]:
similars(warp, toy_movie_id)[:10]

[(1, '0    Toy Story (1995)'),
 (2858, '2789    American Beauty (1999)'),
 (1408, '1385    Last of the Mohicans, The (1992)'),
 (2908, "2839    Boys Don't Cry (1999)"),
 (1552, '1513    Con Air (1997)'),
 (2617, '2548    Mummy, The (1999)'),
 (648, '642    Mission: Impossible (1996)'),
 (1721, '1672    Titanic (1997)'),
 (588, '584    Aladdin (1992)'),
 (924, '912    2001: A Space Odyssey (1968)')]

In [49]:
recommend(warp, test_user_id, user_item_csr)[:10]

['2878    Goldfinger (1964)',
 '476    Jurassic Park (1993)',
 '1081    E.T. the Extra-Terrestrial (1982)',
 '1959    Saving Private Ryan (1998)',
 '1220    Terminator, The (1984)',
 '1196    Alien (1979)',
 '1183    Good, The Bad and The Ugly, The (1966)',
 '2882    Fistful of Dollars, A (1964)',
 '1366    Jaws (1975)',
 '3633    Mad Max (1979)']